## Setup

In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge

from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

In [2]:
%env MLFLOW_TRACKING_URI=http://localhost:5000
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ACCESS_KEY_ID=2vSrPs21nZYaUQvovgRL
%env AWS_SECRET_ACCESS_KEY=yCqF29KU1qbykEnsceWMDRNvPelgGAVBmyD6PeU5

env: MLFLOW_TRACKING_URI=http://localhost:5000
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ACCESS_KEY_ID=2vSrPs21nZYaUQvovgRL
env: AWS_SECRET_ACCESS_KEY=yCqF29KU1qbykEnsceWMDRNvPelgGAVBmyD6PeU5


In [3]:
import os
import mlflow

assert "MLFLOW_TRACKING_URI" in os.environ
assert "MLFLOW_S3_ENDPOINT_URL" in os.environ
assert "AWS_ACCESS_KEY_ID" in os.environ
assert "AWS_SECRET_ACCESS_KEY" in os.environ

# mlflow.set_tracking_uri("http://localhost:5000/")
mlflow.set_experiment("nyc-taxi")

<Experiment: artifact_location='s3://mlflow/1', creation_time=1700794101778, experiment_id='1', last_update_time=1700794101778, lifecycle_stage='active', name='nyc-taxi', tags={}>

## Data Preparation

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
train_data_path = "./data/green_tripdata_2021-01.parquet"
val_data_path = "./data/green_tripdata_2021-02.parquet"

df_train = read_dataframe(train_data_path)
df_val = read_dataframe(val_data_path)
len(df_train), len(df_val)

(73908, 61921)

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

## Model Training

In [9]:
import xgboost as xgb

train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [10]:
with mlflow.start_run():
    best_params = {
        "learning_rate": 0.2526805505341685,
        "max_depth": 21,
        "min_child_weight": 1.4699844585464745,
        "objective": "reg:linear",
        "reg_alpha": 0.03181433933064131,
        "reg_lambda": 0.0050476303572755754,
        "seed": 42,
    }

    # log hyperparams
    mlflow.log_params(best_params)

    # train model
    model = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=20,
        evals=[(valid, "validation")],
        early_stopping_rounds=25,
    )

    # evaluate model
    y_pred = model.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("validation-rmse", rmse)

    # log preprocessor artifact
    preprocessor_path = "./models/preprocessor.bin"
    with open(preprocessor_path, "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact(preprocessor_path, artifact_path="preprocessor")

    # log model artifact
    ## cara 1: langsung menggunakan method bawaan (tergantung pada model/library-nya)
    ## dalam hal ini karena pake xgboost ya pake itu juga
    mlflow.xgboost.log_model(model, artifact_path="models")

    ## cara 2: sama seperti pada log preprocessor
    # model_path = "./models/xgboost.bin"
    # with open(model_path, "wb") as f_out:
    #     pickle.dump(dv, f_out)
    # mlflow.log_artifact(model_path, artifact_path="models")

[0]	validation-rmse:10.26015
[1]	validation-rmse:8.94980
[2]	validation-rmse:8.10100
[3]	validation-rmse:7.56022
[4]	validation-rmse:7.21732
[5]	validation-rmse:7.00285
[6]	validation-rmse:6.86549
[7]	validation-rmse:6.77423
[8]	validation-rmse:6.71627
[9]	validation-rmse:6.67279
[10]	validation-rmse:6.64493
[11]	validation-rmse:6.62168
[12]	validation-rmse:6.60743
[13]	validation-rmse:6.59441
[14]	validation-rmse:6.58689
[15]	validation-rmse:6.58038
[16]	validation-rmse:6.57834
[17]	validation-rmse:6.57420
[18]	validation-rmse:6.57302
[19]	validation-rmse:6.57114


## Using Model Artifact

In [11]:
logged_model = "runs:/9d249fc0e2a7422e8d587b6123e8c638/models"

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.xgboost
  run_id: 9d249fc0e2a7422e8d587b6123e8c638

In [12]:
loaded_model.predict(X_val[:10])

array([16.294361 ,  7.2582555, 19.246721 , 24.319605 ,  9.923026 ,
       17.105246 , 13.982095 ,  8.828904 ,  8.960282 , 18.456018 ],
      dtype=float32)

In [13]:
# load model as xgboost model
xgboost_loaded_model = mlflow.xgboost.load_model(logged_model)
xgboost_loaded_model

In [14]:
pred = xgboost_loaded_model.predict(valid)
pred[:10]

array([16.294361 ,  7.2582555, 19.246721 , 24.319605 ,  9.923026 ,
       17.105246 , 13.982095 ,  8.828904 ,  8.960282 , 18.456018 ],
      dtype=float32)

## What i learned

- Konsep model management yang merupakan bagian dari MLOps yang berisi experiment tracking, model versioning, dan model deployment (lengkap lihat di slides)
- Kenapa kita perlu? Mengurangi error prone, no versioning, dan no model lineage
- Model atau file yang disimpan pada experiment run di MLflow disebut dengan artifact
- Untuk menyimpan model sebagai artifact ada dua cara yaitu:
  - Log model as an artifact: <br>
  `mlflow.log_artifact("mymodel", artifact_path="models/")`
  - Log model using the method `log_model` (tergantung support mlflow terhadap suatu library): <br> 
  `mlflow.<framework>.log_model(model, artifact_path="models/")`
- Begitu juga dengan cara menggunakan artifact terdapat dua cara yaitu:
  - Load sebagai MLflow PyFuncModel
  - Load sebagai model dari library/framework itu sendiri